In [ ]:
import pandas as pd
from dash import Dash, Input, Output, callback, dcc, html, State
import plotly.express as px
import dash_bootstrap_components as dbc

df = pd.read_csv(r"C:\Workflow\SSML_Data\datasets\primary_dashboard.csv")
df = df[df['primary_lifting']>0]
df = df.dropna()


app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container(
    [
        html.H4(
            "Region Wise Primary Lifting",
            style={"textAlign": "center"},
            className="mb-3",
        ),
        # ------------------------------------------------- #
        # Modal
        html.Div(
            [
                dbc.Button("State Seelection", id="open", n_clicks=0),
                dbc.Modal(
                    [
                        dbc.ModalHeader(dbc.ModalTitle("Filters")),
                        dbc.ModalBody(
                            [
                                # Filter within dbc Modal
                                html.Label("state"),
                                dcc.Dropdown(
                                    id="dynamic_callback_dropdown_region",
                                    options=[
                                        {"label": x, "value": x}
                                        for x in sorted(df["state"].unique())
                                    ],
                                    multi=True,
                                ),
                                html.Br(),
                                html.Label("Ship Date"),
                                dcc.DatePickerRange(
                                    id="my-date-picker-range",
                                    min_date_allowed=min(df["ship_date"]),
                                    max_date_allowed=max(df["ship_date"]),
                                    end_date=max(df["ship_date"]),
                                    start_date=min(df["ship_date"]),
                                    clearable=True,
                                ),
                            ]
                        ),
                    ],
                    id="state selection",
                    is_open=False,
                ),
            ],
            className="mb-5",
        ),
        # ---------------------------------------- #
        # Tabs
        dcc.Tabs(
            id="tab",
            value="treemap",
            children=[
                dcc.Tab(label="Treemap", value="treemap"),
                dcc.Tab(label="Sunburst", value="sunburst"),
            ],
        ),
        html.Div(id="tabs-content"),
    ],
    fluid=True,
)


@callback(
    Output("tabs-content", "children"),
    Input("dynamic_callback_dropdown_region", "value"),
    Input("tab", "value"),
    Input("my-date-picker-range", "start_date"),
    Input("my-date-picker-range", "end_date"),
)
def main_callback_logic(region, tab, start_date, end_date):
    dff = df.copy()

    if region is not None and len(region) > 0:
        dff = dff.query("Region == @region")

    if start_date is not None:
        dff = dff.query("ship_date > @start_date")

    if end_date is not None:
        dff = dff.query("ship_date < @end_date")

    dff = dff.groupby(by=["Segment", "Region"]).sum().reset_index()

    if tab == "treemap":
        fig = px.treemap(
            dff, path=[px.Constant("all"), "Segment", "Region"], values="profit_derived"
        )
    else:
        fig = px.sunburst(
            dff, path=[px.Constant("all"), "Segment", "Region"], values="profit_derived"
        )

    fig.update_traces(root_color="lightgrey")
    fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))

    return dcc.Graph(figure=fig)


@callback(
    Output("modal", "is_open"),
    Input("open", "n_clicks"),
    State("modal", "is_open"),
)
def toggle_modal(n1, is_open):
    if n1:
        return not is_open
    return is_open


if __name__ == "__main__":
    app.run_server()